In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


In [8]:
TRAIN_DIR = "/content/drive/MyDrive/CAT_DATASET_INFOSYS/train"
VALID_DIR = "/content/drive/MyDrive/CAT_DATASET_INFOSYS/valid"


In [9]:
import os
print(os.listdir(TRAIN_DIR))
print(os.listdir(VALID_DIR))


['Surprised', 'Scared', 'Sad', 'Disgusted', 'Normal', 'Happy', 'Angry']
['Surprised', 'Scared', 'Happy', 'Sad', 'Normal', 'Angry', 'Disgusted']


In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (160, 160)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    VALID_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)


Found 502 images belonging to 7 classes.
Found 169 images belonging to 7 classes.


In [12]:
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(160, 160, 3)
)

base_model.trainable = False  # IMPORTANT

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
outputs = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=outputs)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [13]:
from tensorflow.keras.optimizers import Adam

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [14]:
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=20
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 261s 15s/step - accuracy: 0.2233 - loss: 2.2017 - val_accuracy: 0.5266 - val_loss: 1.4679
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 216ms/step - accuracy: 0.5553 - loss: 1.2148 - val_accuracy: 0.5917 - val_loss: 1.2476
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 261ms/step - accuracy: 0.7068 - loss: 0.9773 - val_accuracy: 0.5740 - val_loss: 1.2028
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 210ms/step - accuracy: 0.7269 - loss: 0.7429 - val_accuracy: 0.6036 - val_loss: 1.1439
Epoch 5/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 211ms/step - accuracy: 0.8612 - loss: 0.5019 - val_accuracy: 0.6095 - val_loss: 1.1103
Epoch 6/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 217ms/step - accuracy: 0.9188 - loss: 0.3905 - val_accuracy: 0.5799 - val_loss: 1.1514
Epoch 7/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 248ms/step - accuracy: 0.9439 - loss: 0.2951 - val_accuracy: 0.6331 - val_loss: 1.1279
Epoch 8/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 213ms/step - accuracy: 0.9562 - loss: 0.2526 - val_accuracy: 0.

In [15]:
for layer in base_model.layers[-20:]:
    layer.trainable = True


In [16]:
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [17]:
history_finetune = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=10
)


Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.6375 - loss: 1.2467 - val_accuracy: 0.6036 - val_loss: 1.3813
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 213ms/step - accuracy: 0.6973 - loss: 1.0823 - val_accuracy: 0.5917 - val_loss: 1.4259
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 217ms/step - accuracy: 0.7385 - loss: 0.7114 - val_accuracy: 0.5799 - val_loss: 1.4625
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 247ms/step - accuracy: 0.7637 - loss: 0.6788 - val_accuracy: 0.5799 - val_loss: 1.4912
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 232ms/step - accuracy: 0.8310 - loss: 0.5043 - val_accuracy: 0.5740 - val_loss: 1.5129
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 213ms/step - accuracy: 0.8569 - loss: 0.4096 - val_accuracy: 0.5680 - val_loss: 1.5335
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 219ms/step - accuracy: 0.8807 - loss: 0.3664 - val_accuracy: 0.5680 - val_loss: 1.5455
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 261ms/step - accuracy: 0.8776 - loss: 0.3325 - val_accuracy: 0.57